In [1]:
import pandas as pd
import numpy as np
from pandas import Series, DataFrame

In [2]:
import os

In [4]:
os.getcwd()

'/Users/journnie/Desktop/asia/aepython/machine_learning/300.Classification'

In [9]:
titanic_df = pd.read_csv("data/titanic_train.csv")

In [10]:
titanic_df.head()
#passenger ID: 탑싀객 일련번호
#Survived: 생존여부(label) : 0: 사망, 1:생존
# Pclass: 선실 등급
# SibSp:  동승한 형제자매 혹은 배우자 인원수
#Parch:  동승한 부모님 또는 자녀수
# Ticket: 티켓 번호
#Embaked: 중간 정착 항구: C: Cherbourg, Q= Queenstown, S=Southhampton

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [ ]:
#0) 전처리

In [11]:
#일단 한 번 해보기
# target(label): Survived
# feature: 나머지 컬럼
X_titanic_df = titanic_df.drop('Survived', axis=1)
y_titanic_df = titanic_df.Survived

In [12]:
#문자열 피처를 삭제(문자열이 있으면 학습할 수 없음)
X_titanic_df.drop(['PassengerId','Name','Ticket','Cabin'],axis=1,inplace=True)
X_titanic_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    891 non-null    int64  
 1   Sex       891 non-null    object 
 2   Age       714 non-null    float64
 3   SibSp     891 non-null    int64  
 4   Parch     891 non-null    int64  
 5   Fare      891 non-null    float64
 6   Embarked  889 non-null    object 
dtypes: float64(2), int64(3), object(2)
memory usage: 48.9+ KB


In [13]:
titanic_df.Embarked.value_counts()

Embarked
S    644
C    168
Q     77
Name: count, dtype: int64

In [ ]:
# null값 처리, string 처리필요

In [14]:
#2) 문자열 피처 인코딩(숫자값으로 변환)
# 성별 피처를 LabelEncoding하여 숫자로 변환
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
X_titanic_df['Sex'] = le.fit_transform(X_titanic_df.Sex)

In [15]:
#Embarked 피처를 onehot Encoding하여 숫자로 변환
#Enbarked에는 Null값이 2개 있음 - 처리요망
X_titanic_df.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,1,22.0,1,0,7.2500,S
1,1,0,38.0,1,0,71.2833,C
2,3,0,26.0,0,0,7.9250,S
3,1,0,35.0,1,0,53.1000,S
4,3,1,35.0,0,0,8.0500,S


In [16]:
X_titanic_df= pd.get_dummies(X_titanic_df)

X_titanic_df.isnull().sum()

Pclass          0
Sex             0
Age           177
SibSp           0
Parch           0
Fare            0
Embarked_C      0
Embarked_Q      0
Embarked_S      0
dtype: int64

In [17]:
#3) NaN  결측치 채워넣기 (Age 항목 714개)
X_titanic_df.Age.fillna(X_titanic_df.Age.mean(),inplace=True)
X_titanic_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Pclass      891 non-null    int64  
 1   Sex         891 non-null    int64  
 2   Age         891 non-null    float64
 3   SibSp       891 non-null    int64  
 4   Parch       891 non-null    int64  
 5   Fare        891 non-null    float64
 6   Embarked_C  891 non-null    bool   
 7   Embarked_Q  891 non-null    bool   
 8   Embarked_S  891 non-null    bool   
dtypes: bool(3), float64(2), int64(4)
memory usage: 44.5 KB


In [18]:
X_titanic_df.info() #Age 값의 결측치 확인

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Pclass      891 non-null    int64  
 1   Sex         891 non-null    int64  
 2   Age         891 non-null    float64
 3   SibSp       891 non-null    int64  
 4   Parch       891 non-null    int64  
 5   Fare        891 non-null    float64
 6   Embarked_C  891 non-null    bool   
 7   Embarked_Q  891 non-null    bool   
 8   Embarked_S  891 non-null    bool   
dtypes: bool(3), float64(2), int64(4)
memory usage: 44.5 KB


In [19]:
# Age columms을 Age_group컬럼으로 변환
# 0 ~ 5 : Baby, 6~JK: Child, 13~18: Teenager, 19~25: Student, 26~35: Young Adult, 36~60: Adult, 61~: elderly
np.digitize(X_titanic_df.Age,[0,6,13,19,26,36,61])

array([4, 6, 5, 5, 5, 5, 6, 1, 5, 3, 1, 6, 4, 6, 3, 6, 1, 5, 5, 5, 5, 5,
       3, 5, 2, 6, 5, 4, 5, 5, 6, 5, 5, 7, 5, 6, 5, 4, 3, 3, 6, 5, 5, 1,
       4, 5, 5, 5, 5, 3, 2, 4, 6, 5, 7, 5, 4, 5, 1, 2, 4, 6, 6, 1, 5, 5,
       5, 4, 3, 5, 5, 3, 4, 5, 5, 4, 5, 5, 1, 5, 4, 5, 5, 5, 3, 5, 3, 5,
       4, 4, 5, 4, 6, 5, 6, 5, 7, 4, 5, 5, 5, 5, 4, 5, 6, 5, 4, 5, 6, 5,
       6, 3, 4, 4, 3, 4, 7, 5, 4, 1, 4, 5, 5, 5, 6, 2, 5, 4, 5, 6, 5, 4,
       6, 5, 4, 4, 4, 6, 3, 4, 5, 4, 4, 4, 3, 4, 5, 2, 6, 6, 6, 4, 6, 6,
       5, 6, 3, 5, 5, 5, 6, 6, 5, 3, 1, 2, 5, 6, 5, 5, 7, 1, 1, 4, 6, 3,
       5, 6, 5, 6, 5, 5, 2, 1, 1, 5, 5, 6, 6, 6, 5, 4, 4, 1, 6, 6, 5, 6,
       5, 4, 5, 5, 5, 6, 3, 1, 5, 5, 3, 6, 4, 5, 4, 5, 5, 5, 5, 6, 5, 5,
       3, 5, 6, 5, 6, 4, 4, 4, 3, 5, 5, 5, 6, 1, 4, 5, 6, 2, 4, 5, 5, 5,
       5, 4, 5, 6, 4, 4, 6, 6, 5, 5, 7, 5, 6, 5, 5, 5, 5, 6, 5, 1, 6, 6,
       5, 6, 3, 4, 6, 5, 5, 4, 6, 6, 5, 7, 6, 5, 2, 5, 7, 5, 3, 4, 5, 5,
       5, 4, 6, 4, 5, 4, 6, 4, 4, 5, 4, 1, 5, 6, 5,

In [20]:
X_titanic_df['Age_group'] = np.digitize(X_titanic_df.Age,[0,6,13,19,26,36,61])
X_titanic_df

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S,Age_group
0,3,1,22.000000,1,0,7.2500,False,False,True,4
1,1,0,38.000000,1,0,71.2833,True,False,False,6
2,3,0,26.000000,0,0,7.9250,False,False,True,5
3,1,0,35.000000,1,0,53.1000,False,False,True,5
4,3,1,35.000000,0,0,8.0500,False,False,True,5
...,...,...,...,...,...,...,...,...,...,...
886,2,1,27.000000,0,0,13.0000,False,False,True,5
887,1,0,19.000000,0,0,30.0000,False,False,True,4
888,3,0,29.699118,1,2,23.4500,False,False,True,5
889,1,1,26.000000,0,0,30.0000,True,False,False,5


In [21]:
## Age와 Age_group 겹치므로 Age drop
X_titanic_df.drop("Age",axis=1,inplace=True)

In [22]:
#1) 학습용, 테스트용 데이터 분리
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X_titanic_df,y_titanic_df,test_size=0.2,random_state=1)

In [23]:
#2) 결정트리 객체 생성
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=42)

In [24]:
#3) 학습
dt.fit(X_train,y_train)

DecisionTreeClassifier(random_state=42)

In [25]:
#4) 평가
train_score = dt.score(X_train,y_train)
test_score  = dt.score(X_test,y_test)
print(f"train_score : {train_score}")
print(f"test_score :{test_score}")

train_score : 0.949438202247191
test_score :0.776536312849162


In [ ]:
# 평가 점수가 낮아 전처리를 세분화해보기로 함


In [26]:
#결정트리 최적화 - 하이퍼파라미터 튜닝 & 교차검증 -: GridSearchCV()
from sklearn.model_selection import GridSearchCV

In [27]:
params = {
    'max_depth': [3, 5, 7],
    'min_samples_split' : [5,10,15],
    'splitter': ['best','random']
}

In [28]:
gs = GridSearchCV(dt,param_grid=params,cv=5)
gs.fit(X_train,y_train)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(random_state=42),
             param_grid={'max_depth': [3, 5, 7],
                         'min_samples_split': [5, 10, 15],
                         'splitter': ['best', 'random']})

In [29]:
best_dt = gs.best_estimator_
best_dt

DecisionTreeClassifier(max_depth=3, min_samples_split=5, random_state=42,
                       splitter='random')

In [30]:
## 점수는 낮아졌지만 폭이 줄어들어서 신뢰도 증가.
print(best_dt.score(X_train,y_train))
print(best_dt.score(X_test,y_test))

0.8384831460674157
0.7541899441340782


In [31]:
pd.DataFrame(gs.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_min_samples_split,param_splitter,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.001452,0.000305,0.000770,0.000104,3,5,best,"{'max_depth': 3, 'min_samples_split': 5, 'spli...",0.818182,0.832168,0.852113,0.866197,0.830986,0.839929,0.017039,4
1,0.001235,0.000137,0.000776,0.000121,3,5,random,"{'max_depth': 3, 'min_samples_split': 5, 'spli...",0.818182,0.853147,0.838028,0.873239,0.830986,0.842716,0.018984,1
2,0.001149,0.000088,0.000680,0.000070,3,10,best,"{'max_depth': 3, 'min_samples_split': 10, 'spl...",0.818182,0.832168,0.852113,0.866197,0.830986,0.839929,0.017039,4
3,0.000928,0.000097,0.000597,0.000046,3,10,random,"{'max_depth': 3, 'min_samples_split': 10, 'spl...",0.818182,0.853147,0.838028,0.873239,0.830986,0.842716,0.018984,1
4,0.001058,0.000124,0.000614,0.000048,3,15,best,"{'max_depth': 3, 'min_samples_split': 15, 'spl...",0.818182,0.832168,0.852113,0.866197,0.830986,0.839929,0.017039,4
5,0.000867,0.000014,0.000572,0.000012,3,15,random,"{'max_depth': 3, 'min_samples_split': 15, 'spl...",0.818182,0.853147,0.838028,0.873239,0.830986,0.842716,0.018984,1
6,0.001119,0.000073,0.000608,0.000062,5,5,best,"{'max_depth': 5, 'min_samples_split': 5, 'spli...",0.825175,0.832168,0.859155,0.852113,0.809859,0.835694,0.017947,9
7,0.000917,0.000026,0.000568,0.000012,5,5,random,"{'max_depth': 5, 'min_samples_split': 5, 'spli...",0.818182,0.818182,0.830986,0.859155,0.781690,0.821639,0.024958,15
8,0.001102,0.000092,0.000585,0.000039,5,10,best,"{'max_depth': 5, 'min_samples_split': 10, 'spl...",0.825175,0.832168,0.859155,0.866197,0.809859,0.838511,0.021127,7
9,0.000895,0.000010,0.000564,0.000009,5,10,random,"{'max_depth': 5, 'min_samples_split': 10, 'spl...",0.804196,0.811189,0.838028,0.866197,0.802817,0.824485,0.024418,11


In [32]:
#다른 분류 알고리듬으로 예측하기 (나중에 배우고 해봐야 할듯)
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

svc = SVC()
lr = LogisticRegression()

In [33]:
%matplotlib inline

In [ ]:
# 결정 트리 시각화

In [ ]:
%matplotlib inline

In [ ]:
#결정트리 시각화

In [ ]:
#피처와 라벨 정보를 기입하여 가독성 향상


In [ ]:
# 간단한 트리 생성

In [ ]:
#결정트리 그래프를 파일로 받아 Pdf 저장
